In [2]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7
!pip install pypdf2
!pip install transformers
!pip install torch
!playwright install
!playwright install-deps
!pip install unstructured[pdf]
!pip install python-docx
!pip install python-pptx
!pip install gradio

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\asifhussain\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\asifhussain\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\asifhussain\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\asifhussain\anaconda3\lib\ssl.py", line 1242, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\asifhussain\anaconda3\lib\ssl.py", line 1100, in re

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\asifhussain\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\asifhussain\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\asifhussain\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\asifhussain\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\asifhussain\anaconda3\lib\ssl.py", line 1242, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\asifhussain\anaconda3\lib\ssl.py", line 1100, in re

'playwright' is not recognized as an internal or external command,
operable program or batch file.
'playwright' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Could not find a version that satisfies the requirement unstructured[pdf] (from versions: none)
ERROR: No matching distribution found for unstructured[pdf]


ERROR: Could not find a version that satisfies the requirement gradio (from versions: none)
ERROR: No matching distribution found for gradio


In [7]:
!pip install python-pptx

  Using cached python_pptx-0.6.23-py3-none-any.whl (471 kB)
  Using cached XlsxWriter-3.2.0-py3-none-any.whl (159 kB)


In [2]:
!pip install gradio

In [3]:
# parser

# PDF
import PyPDF2
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_file_path):
    try:

      reader = PdfReader(pdf_file_path)
      page = reader.pages[0]

      pages = []
      for test in reader.pages :
        # print(test.extract_text())
        pages.append(test.extract_text())
      print(type(page))

      extracted = page.extract_text()
      # return extracted

      document = ''.join(pages)

      ans = []
      check = False
      first = True
      for char in document :
        if char == '[' or check :
          check = True
        else :
          if char.isalnum() or char == ' ' or char == '.' or char == '\n' :
            ans.append(char)
        if char == ']' :
          check = False
        # if first and len(ans) > 8 and ''.join(ans[-8:]).upper() == "ABSTRACT":
        #   ans = ['A','B','S','T','R','A','C','T']
        #   # print("INSIDE")
        #   first = False

      # print(''.join(ans[-8:]))
      ans = ''.join(ans)

      ans = ans.lower()

      #  Consider changing cases
      if "abstract" in ans:
        abs = ans.index("abstract")
      else : abs = 0
      if "references" in ans:
        x = ans.rindex("references")
      else : x = len(ans)-1
      useful_text = ans[:]
      return useful_text

    except Exception as e:
        print(f"Error: {e}")

In [4]:
from docx import Document

def is_textual_run(run):
    # Check if a run contains non-empty textual content (excluding images, etc.)
    return run.text.strip()

def is_valid_paragraph(paragraph):
    # Check if a paragraph contains valid textual content (excluding images, graphs, etc.)
    return any(is_textual_run(run) for run in paragraph.runs)

def extract_text_from_docx(docx_path):
    # Create a Document object
    doc = Document(docx_path)

    # Initialize an empty string to store extracted text
    extracted_text = ""

    # Iterate through paragraphs and add valid text to the string
    for paragraph in doc.paragraphs:
        if is_valid_paragraph(paragraph):
            extracted_text += paragraph.text + "\n"

    return extracted_text


In [8]:
# pptx

import re
from pptx import Presentation

def extract_text_from_shape(shape):
    text = ""
    if shape.has_text_frame:
        for paragraph in shape.text_frame.paragraphs:
            text += paragraph.text + "\n"
    return text

def extract_text_from_pptx(ppt_file):
    presentation = Presentation(ppt_file)
    text = ""

    for slide in presentation.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += extract_text_from_shape(shape)
            elif shape.has_table:
                table = shape.table
                for row in table.rows:
                    for cell in row.cells:
                        text += extract_text_from_shape(cell)

    # Remove multiple spaces while preserving '\n'
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()  # Remove leading and trailing spaces

In [9]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
# from langchain.document_transformers import Html2TextTransformer
# from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [10]:
# Chunk text

def chunker(research_paper):
  text_splitter = CharacterTextSplitter(
  separator = "\n",
  chunk_size = 512,
  chunk_overlap = 50,
  length_function = len,
  )

  chunked_documents = text_splitter.split_text(research_paper)
  # print(ans)
  # size = 0
  # for i in chunked_documents: size += len(i)
  # print(size)
  # print(len(chunked_documents))

  # Load chunked documents into the FAISS index
  db = FAISS.from_texts(chunked_documents,
                            HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

  retriever = db.as_retriever()

  return retriever

In [18]:
# model

def define_model(retriever):

  model_name='mistralai/Mistral-7B-Instruct-v0.1'

  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"

  # Activate 4-bit precision base model loading
  use_4bit = True

  # Compute dtype for 4-bit base models
  bnb_4bit_compute_dtype = "float16"

  # Quantization type (fp4 or nf4)
  bnb_4bit_quant_type = "nf4"
  # bnb_4bit_quant_type = "fp4"

  # Activate nested quantization for 4-bit base models (double quantization)
  use_nested_quant = False
  # use_nested_quant = True

  compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=use_4bit,
      bnb_4bit_quant_type=bnb_4bit_quant_type,
      bnb_4bit_compute_dtype=compute_dtype,
      bnb_4bit_use_double_quant=use_nested_quant,
  )

  # Check GPU compatibility with bfloat16
  if compute_dtype == torch.float16 and use_4bit:
      major, _ = torch.cuda.get_device_capability()
      if major >= 8:
          print("=" * 80)
          print("Your GPU supports bfloat16: accelerate training with bf16=True")
          print("=" * 80)

  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
  )

  # return tokenizer, model
  text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
  )

  mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

  # return mistral_llm
  prompt_template = """
  ### [INST] Instruction: Answer the question based only based knowledge by given context. Here is context to help:

  {context}

  ### QUESTION:
  {question} [/INST]
  """

  # Create prompt from prompt template
  prompt = PromptTemplate(
      input_variables=["context", "question"],
      template=prompt_template,
  )

  # Create llm chain
  llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

  rag_chain = ( {"context": retriever, "question": RunnablePassthrough()} | llm_chain )

  return rag_chain

In [12]:
# function that will identify
def file_extension(file_list):
  data = ""
  for file in file_list:
      if file.endswith('.pptx') :
          print('ppt or pptx file')
          data = extract_text_from_pptx(file)
      elif file.endswith('.pdf'):
          print('pdf file')
          data = extract_text_from_pdf(file)
      elif file.endswith('.docx') :
          print('doc or docx file')
          data = extract_text_from_docx(file)
      # elif file.endswith('.dox') or file.endswith('.ppt'):
      #     print("Unstructured file")
      else:
          data = "Sorry"

  return data

In [13]:
import gradio as gr

filenames = []

def upload_files(files):
    global filenames
    filenames = [files]
    file_extension(filenames)
    return "file uploaded successfully"

inputs = [gr.File(label="Upload File", file_count="single", file_types=[".pdf", ".pptx", ".docx"])]
output = [gr.Textbox(label="Uploaded Files")]
# , gr.Button(fn=start_parsing)

interface = gr.Interface(fn=upload_files, inputs=inputs, outputs=output, title="File Uploader")
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c253dcf43a2195ca34.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [19]:
print(filenames)
data = file_extension(filenames)
ret = chunker(data)
rag_chain = define_model(ret)


['/tmp/gradio/1fb2db931b28cf4b673bf5344b20bbd88474645f/test.pdf']
pdf file
<class 'PyPDF2._page.PageObject'>


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [20]:
result = rag_chain.invoke("What is paragraph level formating?")

print(result["text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Paragraph level formatting refers to the application of formatting options to entire paragraphs of text, rather than just individual words or phrases. This can include things like font size, color, and style, as well as alignment and spacing. In the given context, paragraph level formatting is used to apply different styles to different paragraphs of text, such as bold and italicized text, different colors, and hanging indents.


In [21]:
import gradio as gr

def echo(message, history):
    result = rag_chain.invoke(message)
    return result['text']
    # return "No Output"

demo = gr.ChatInterface(fn=echo, title="Chat with PDF", retry_btn=None, undo_btn=None)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dcfc59147276ad3d21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
